<a href="https://colab.research.google.com/github/abdulrahmaninfra/gu/blob/main/Galala_Bank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Project Initialization & Data Structure
Explanation: In this cell, we import the necessary libraries. We use json to store user data permanently and os to interact with the system. We also define a 2D List called user_data to act as our initial database. To make the code more readable, we assign meaningful names (constants) to the list indices.

In [ ]:
import os
import json

# Database using a list of lists (2D list) [Account Number, PIN, Balance, Name, Is Admin]
user_data = [
    ["101", "1111", 4000, "Ahmed", False],
    ["102", "2222", 2000, "AbdelRahman", False],
    ["103", "3333", 2000, "Zeyad", False],
    ["104", "4444", 2000, "Moaz", False],
    ["105", "5555", 3000, "Mohamed", False],
    ["999", "0000", 0, "Admin", True]
]

# Constants for easier indexing
acc_number = 0
user_pin = 1
balance = 2
name = 3
is_admin = 4

# 2. Storage & UI Management
Explanation
This part of the code handles two critical tasks:

Persistence (The Memory): Since variables are temporary, we use json to save user_data into a permanent file on your HP Elitebook's drive. This ensures that even if you restart your computer, your balances and account changes are still there.

UI Cleanup (The Visuals): We use os.system to "talk" to your operating system's terminal. It clears out old text so the user only sees the current menu, making it look like a real ATM software instead of a long list of text.

Functional Breakdown
save_user_data(): Opens user_data.json in "write" mode and dumps the Python list into it.

load_user_data(): Uses os.path.exists to check if the file exists. If it does, it "reads" the file and updates the program's memory.

clear(): Uses os.name to detect if you are on Windows ('nt') or Linux ('posix') and sends the correct "clear" command (cls or clear).

In [ ]:
def save_user_data():

    with open("user_data.json", "w") as file:
        json.dump(user_data, file)


def load_user_data():

    global user_data
    if os.path.exists("user_data.json"):
        with open("user_data.json", "r") as file:
            user_data = json.load(file)

def clear():
    if os.name == "nt":
        os.system("cls")
    else:
        os.system("clear")


# 3. Authentication System
Explanation: The login() function is the gatekeeper. It prompts the user for their account number and PIN. It then iterates through the user_data list to find a match. If a match is found, it returns the specific user's data; otherwise, it returns None.

In [ ]:
def login():  # take account number and pin as input from user
    account_num = input("Enter your account number: ").strip()
    pin = input("enter your PIN: ").strip()
    for current_user in user_data:
        if current_user[acc_number] == account_num and current_user[user_pin] == pin:
            return current_user
    print("Invalid account number or PIN")
    return None

# 4. Financial Operations
Explanation: These functions handle the core banking logic for regular users:

Deposit: Increases the user's balance.

Withdraw: Decreases the balance after checking if the user has sufficient funds.

Transfer: Finds a target account and moves money from the current user to the recipient.

Check Balance: Simply displays the current amount of money in the account.

In [ ]:
def deposit(current_user):
        amount = float(input("Enter a amount to Deposit: ").strip())
        if amount <= 0:
            print("\nInvalid amount. Please enter a positive value.")
            return
        current_user[balance] += amount
        save_user_data()
        print(f"\nNew Balance: {current_user[balance]}")

def withdraw(current_user):
        amount = float(input("Enter amount to Withdraw: ").strip())
        if amount <= 0:
            print("\nInvalid amount. Please enter a positive value.")
            return
        if amount <= current_user[balance]:
            current_user[balance] -= amount
            save_user_data()
            print(f"\nWithdrawal of {amount} successful. Remaining: ${current_user[balance]}")
        else:
            print("No enough balance!")



def transfer(current_user):
        target_account = input("Enter the account number want to transfer to: ").strip()

        if target_account == current_user[acc_number]:
            print("You cannot transfer to your own account!")
            return

        amount = float(input("Enter amount to Transfer: ").strip())
        if amount <= 0:
            print("Invalid amount. Please enter a positive value.")
            return
        target_found = False
        for user in user_data:
            if target_account == user[acc_number]:
                target_found = True
                if amount <= current_user[balance]:
                    current_user[balance] -= amount
                    user[balance] += amount
                    save_user_data()
                    print(f"\nTransfer of ${amount} to {user[name]} successful. Your Remaining Balance: ${current_user[balance]}")
                else:
                    print(f"No enough balance! Your Balance: ${current_user[balance]}")
                    break
        if not target_found:
            print("User's account number not found.")
            return



def check_balance(current_user):
        print(f"\nYour current balance is: ${current_user[balance]}")

# 5. Administrative Controls
Explanation: If the logged-in user is an Admin, they get access to these management tools. They can create new accounts, change any user's PIN or balance, and view a master list of all accounts in the system.

In [ ]:
def add_user():  # Add User Function
    new_user_account_number = input("\nThe new account number: ").strip()
    for user in user_data:
        if new_user_account_number == user[acc_number]:
            print("This account number is already exist. Please choose another one.")
            return
    new_user_pin = input("\nEnter the new user's PIN: ").strip()
    new_user_balance = float(input("\nEnter the new user's balance: ").strip())
    new_user_name = input("\nEnter the new user's name: ").strip()
    user_data.append(
        [
            new_user_account_number,
            new_user_pin,
            new_user_balance,
            new_user_name,
            False,
        ]
    )
    save_user_data()
    print("User added successfully.")

def manage_pin():  # Manage Pin
    user_change_pin = input("\nEnter the user's account number: ")
    found = False
    for user in user_data:
        if user_change_pin == user[acc_number]:
            user_new_pin = input("\nEnter the new PIN: ")
            user[user_pin] = user_new_pin
            save_user_data()
            print(f"\nThe new PIN is: {user_new_pin} ")
            found = True
            break
    if not found:
        print("error: user not found")

def manage_balance():  # Manage Balance
        user_change_balance = input("\nEnter the user's account number: ")
        found = False
        for user in user_data:
            if user_change_balance == user[acc_number]:
                user_new_balance = float(input("\nEnter the new balance: "))
                user[balance] = user_new_balance
                save_user_data()
                print(f"Your new balance is: {user_new_balance}")
                found = True
                break
        if not found:
            print("error: user not found")

def view_all_users():
    for user in user_data:
        print(f"Account Number: {user[acc_number]} || Name: {user[name]} || Balance: ${user[balance]} || Is Admin: {user[is_admin]}")

# 6. Main Program Loop
Explanation: This is the entry point of the application. It first loads existing data, then enters a While loop to keep the program running. It handles the logic of switching between the Financial Menu (for customers) and the Admin Menu (for staff).

In [ ]:
def financial_menu(current_user):  # Combine each of the following functions for financial operations (Deposit, Withdraw, Transfer, Check Balance)
    while True:
        clear()
        print(f"{'='*(26+len(current_user[name]))}")
        print(f"MAIN MENU: Welcome Back! {current_user[name]}")
        print(f"{'='*(26+len(current_user[name]))}")
        print("1. Deposit")
        print("2. Withdraw")
        print("3. Transfer")
        print("4. Check Balance")
        print("5. Exit")
        choice = input("\nChoose an option: ").strip()
        if choice == "1":
            deposit(current_user)
        elif choice == "2":
            withdraw(current_user)
        elif choice == "3":
            transfer(current_user)
        elif choice == "4":
            check_balance(current_user)
        elif choice == "5":
            print("you have exited.")
            break
        else:
            print("Invalid choice.")
            return
        input("Press Enter to continue...")


def admin_menu(current_user):
    while True:
        clear()
        print(f"{'='*(29+len(current_user[name]))}")
        print(f"ADMIN MENU: Welcome Back! {current_user[name]}")
        print(f"{'='*(29+len(current_user[name]))}")
        print("1. Add user")
        print("2. Manage pin")
        print("3. Manage balance")
        print("4. Show all users")
        print("5. Exit")

        admin_choice = input("Enter your choice: ").strip()
        if admin_choice == "1":
            add_user()
        elif admin_choice == "2":
            manage_pin()
        elif admin_choice == "3":
            manage_balance()
        elif admin_choice == "4":
            view_all_users()
        elif admin_choice == "5":
            print("you have exited.")
            clear()
            break
        else:
            print("Invalid choice.")
            return
        input("Press Enter to continue...")

def main():
    load_user_data()
    while True:
        clear()
        print(f"{'='*26}")
        print("  WELCOME TO GALALA BANK")
        print(f"{'='*26}")
        active_user = login()

        if active_user:
            if active_user[is_admin]:
                admin_menu(active_user)
            else:
                financial_menu(active_user)
        else:
            retry = input("Try again? (y/n): ").lower().strip()
            if retry != 'y':
                print("Goodbye!")
                break

main()


Admin -->
Account Number: 999
Password: 0000

Ahmed -->
Account Number: 101
Password: 1111

AbdelRahman -->
Account Number: 102
Password: 2222

Zeyad -->
Account Number: 103
Password: 3333

Moaz -->
Account Number: 104
Password: 4444

Mohamed -->
Account Number: 105
Password: 5555

